### __Basic Terminologies__

    - Docker Image: Image is a file which contains OS, software and appliation information. In very simple terms docker image is just like an ISO image file used to install any OS. You can view all docker images(publicly available on DockerHub)
    
    - Docker Container: Container is a running instance of an image. When a Docker Image runs it becomes a Docker Container. You can run the same image again and a different docker container will be created.
    Each container has its own processes, networks and mounts but they share the same OS Kernel.
    
    Containers are not meant to host operating systems but to run a specific task or process.
    
    Whenever a task or process ends in the container, then container exists. A container is alive as long as the task is alive.

    - Dockerfile: A Dockerfile contains all the code to set up a docker container from downloading the docker image to setting the environment.

    Docker file --Build-->Image--Run-->Container

    A single Docker image can be run at different containers.

### __Docker Commands__

    - run: To run a container from an image.
    - ps: List containers and gives basic information about them. -a parameter gives the current status of the containers.
    - stop: Stops a container.
    - rm: Removes the container. 
    - images: Lists images.
    - rmi: Removes an image.
    - pull: Downloads an image.
    - exec: Executes a command in a container.
    - run -attach and detach: Attach mode runs the container and uses the command line. -d paramter allows container to run in detached mode and allows us to use command line.

    - Lab Exercises and Questions

        Q: How to identify running Docker version?
        
        A: Executing 'docker version' at command line 

        Q: How to run and name a container?
        
        A: docker run --name container_name image_name

### __Docker Run__

- To run a specific tag of a software we have to define it at run command: docker run redis:4.0

- To enable stdin to the container we have to enabile interactive mode on terminal: docker run -it image_name

- The underlying host Docker is installed is called Docker Host or Docker Engine. 

- Port mapping: To enable users to reach the applications in the Docker container we need to map the port. With p parameter we can define port to be reached from outer users; docker run -p 80:5000 webapp

- Volume mapping: The applications that procedure data or designed to store data like databases will use containers file system. When we remove the container, data inside the container will also be deleted. 
If we want to mount an external folder to store the data we have to specify it; docker run -v mounted_folder:container_folder docker_image.
In this situation even though we delete the container, data will not be deleted. 

- In order to inspect details about a container; docker inspect container.

- In order to pass environment variable to Docker container; docker run -e ENVIRONMENT_VARIABLE_COLOR webappp_color.

- Lab Exercises and Questions

    Q: How to run Docker container on kodekloud/simple-webapp image with tag blue, host port 38282 and container port is 8080
    
    A: docker run -p 38282:8080 kodekloud/simple-webapp:blue

    Q: How to run an image and passing environment variable to it?
    
    A: docker run -p 38282:8080 --name blue-app -e APP_COLOR=blue -d kodekloud/simple-webapp

    Q: How to run a mysqldb image and passing password to it?
    
    A:docker run --name mysql-db -e MYSQL_ROOT_PASSWORD=db_pass123 mysql

### __Docker Images__

    - How to create my own image?

        First create the Dockerfile;

            FROM Ubuntu <-- Start from a base OS or another image

                Every Docker image must based on another image or OS.

            RUN apt-get update
            RUN apt-get install python

            RUN pip install flask
            RUN pip install flask-mysql

                RUN instruction instructs Docker to run particular command on those based images.

            COPY . /opt/source-code

                COPY instruction copies files from local system into the image.

            ENTRYPOINT FLASK_APP=/opt/source-code/app.py flask run

                ENTRYPOINT creates the entry point to run the image as a container.

        Build the image file;

            docker build Dockerfile -t abc/my-custom-app

            When Docker builds from the image it follows a layered architecture in the Dockerfile which means every statement in the Dockerfile is executed after the previous statement is done. So the size of the image increase cumulatively as the statements run. All the build layers are cached so if any layer fails we don't have to run from the very beginning.

        If the Dockerfile is in the same folder we can build the image like this to the folder that the dockerfile is in;

            docker build -t webapp-color .

        If you want to push it to Dockerhub;

            docker push abc/my-custom-app --> Docker registry

        - Details of Dockerfile;

            Dockerfile has two sections; instruction and argument. Everything on the left side with upper case is an Instruction and everything follows these Instructions is an Argument.

      - Lab Exercises and Questions

            Q: How to understand the OS running in an image?
            A: docker run python:3.6 cat /etc/*release*

### __CMD vs ENTRYPOINT__

    Ultimately, both ENTRYPOINT and CMD give you a way to identify which executable should be run when a container is started from your image. In fact, if you want your image to be runnable (without additional docker run command line arguments) you must specify an ENTRYPOINT or CMD.

    CMD: The main purpose of a CMD is to provide defaults for an executing container.The CMD specifies arguments that will be fed to the ENTRYPOINT.

    ENTRYPOINT: An ENTRYPOINT helps you to configure a container that you can run as an executable. The ENTRYPOINT specifies a command that will always be executed when the container starts.

    Dockerfile;

        FROM Ubuntu

        ENTRYPOINT ["sleep"]

        CMD ["5"]

    docker run ubuntu-sleeper 10 --> takes ENTRYPOINT argument

    docker run ubuntu-sleeper --> takes CMD default


   - Lab Exercises and Questions

       Q: Run ubuntu image and set it sleep for 1000 seconds
       
       A: docker run -d ubuntu sleep 1000

### __Networking in Docker__

  When we install Docker, it creates three networks automatically; Bridge, none and host. 

  Bridge: Bridge is the default network a container gets attached to. It is a private network created by Docker on the host. All containers attached to the Docker get internal IP addresses. Containers can access to each other with using this internal IP. 

  Host: Another way to access the containers externally we can assosiciate the container to host network. This takes out any network isolation between the docker host and docker container. We can not run multiple web containers in the same host with the same port. 

  None: Containers are not attached to any network which means they are on an isolated network. 

  User-defined networks: We can create private networks with;

  docker network create \
      --driver bridge
      --subnet 182.18.0.0/16
      custom-isolated-network

  - To list all the networks;

      docker network ls

  - To inspect Network

      docker inspect docker

          "bridge": {
              "Gateway":
              "IPAddress":
              "MacAddress":
          }

  - Embedded DNS

      - How the containers connect to each other?
          All containers in the docker host can resolve each other with the name of the container. Docker has built in DNS server that hepls the containers to resolve each other by using the container name. 

      - Lab Exercises and Questions    

          Q: How we can inspect the created network to see its settings.
          
          A: docker inspect network_id

          Q: How to run a container with a defined network?
          
          A: docker run --name alpine-2 --network=none alpine

          Q: How to create a private network?
          
          A: docker network create --driver bridge --subnet 182.18.0.1/24 --gateway 182.18.0.1 wp-mysql-network

          Q: How to run a mysqldb by using the created network?
          
          A: docker run --name mysql-db --network=wp-mysql-network -e MYSQL_ROOT_PASSWORD=db_pass123 mysql

          Q: Run container with predefined network with host port?
          
          A: docker run --name webapp -p 38080 -e DB_Host=mysql-db --network=wp-mysql-network kodekloud/simple-webapp-mysql

### __Docker Storage__

    - Docker file system

        /var/lib/docker
            aufs/
            containers/
            image/
            volumes/

    - When we want to create multiple images with similar Dockerfiles which means both have same commands, docker uses cached files to reduce disk usage.

    - Files in the image layer are read only. Files in the container layer are read and write available. 

    - When we want to store the data created by the container, we need to create a volume. Docker automatically creates a volume under the volumes folder.

        docker volume create data_volume

        /var/lib/docker
            volumes/
                data_volume/

    - To run a container with the defined volume;

        docker run -v data_volume:/var/lib/mysql mysql

    - If we forget to create the volume beforehand the run command, Docker automatically creates the volume. This is called volume mounting. 

    - If we want to use an existing volume outer of the docker folders we need to specify it at the run command. This is called bind mounting.

        docker run -v /data/mysql:/var/lib/mysql mysql

    - The new command for mounting the volume is;

         docker run --mount type=bind,source=/data/mysql,target=/var/lib/mysql mysql

    - Docker uses storage drivers to execute these volume specific jobs.

### __Docker Compose__

     When we have multiple containers working together and linked together we have to specify the links, otherwise we will face connection issues. This linking option will be decpreated in the future.

     docker run -d --name=vote -p 5000:80 --link redis:redis voting-app

     When the docker run commands are tested and ready, we can generate the docker-compose.yml file. 

     We will start with dictionary of the container names which we created with run commands. Then we assign the images, links and ports at yml file.

     docker-compose.yml

         redis:
             image: redis
         db:
             image: postgres:9.4
         vote:
             image: voting-app
             ports:
                 - 5000:80
             links:
                 - redis
         result:
             image: result-app
             ports:
                 - 5001:80
             links:
                 - db
         worker:
             image: worker
             links:
                 - redis
                 - db

     With 'docker-compose up' command we bring up the entire application stack.

    This composer version, earliest version, has some issues; selecting different networks is impossible, container dependencies and start up orders are impossible to manage.

    docker-compose.yml

    version: 2
    services:

         redis:
             image: redis
         db:
             image: postgres:9.4
             networks:
                 - back-end
         vote:
             image: voting-app
             networks:
                 - front-end
                 - back-end
             ports:
                 - 5000:80
             depends_on:
                 - redis

    networks:
        front-end:
        back-end:

    - Lab Exercises and Questions

        Q: Next let's create a simple wordpress container called wordpress, image: wordpress, link it to the container db and expose it on host port 8085
        
        A: docker run -d --name=wordpress --link db:db -p 8085:80 wordpress

### __Docker Registry__

    Registry is where all the images are stored.

        image: docker.io/nginx/nginx

            docker.io: registry
            nginx: User/Account
            nginx: Image/Repository

    Private Registry

        If we don't want to share our images, we have to store them at our private registries. All cloud vendors provide this functionality. 


        To use private registry we have to login to the registry.

        docker login private-registry.io

        To run an app in private registry;

        docker run private-registry.io/apps/internal-app

### __Docker Engine__

     - Docker Engine is composed of three parts;

         - Docker CLI: Command line interface to be used at running containers etc.
         - REST API: API interface that programs can talk to the deamon and provide instructions.
         - Docker Deamon: Is a background process that manages docker objects such as images, containers and volumne.

         Docker CLI --> REST API --> Docker Deamon

         Docker CLI can be on a remote location like lap top. We can run a remote container with;
         docker -H=10.123.2.1:2375' nginx

      - Containerization

          Docker uses Namespace to isolate workspace. Process ID, Unix Timesharing, Network, InterProcess and Mount are contained under the Namespace.

          Docker doesn't have own OS, it uses existing host OS. PID is a main term being used to define processes in Linux. Docker needs to distinguish host PIDs and container PIDs. But also container PIDs must live under the host OS. So Container checks for available next PID from OS and creates internal PID with internal numbers but label them with OS PIDs to be seen from the OS.

          - Namespace - PID

              Linux System

                  PID: 1
                      -- PID: 2
                      -- PID: 3
                      -- PID: 4       ---Container---
                      -- PID: 5       PID: 1
                      -- PID: 6          -- PID: 2

          - cgroups(control groups)

              We can restirct a container CPU and Memory usage. Docker uses cgroups to restirct hardware resources to each container. 

              Example below restricts container to use 0.5 of existing CPU at maximum.

              docker run --cpus=.5 ubuntu

              Example below restricts container to use max 100m memory.

              docker run --memory=100m ubuntu

              Linux System

              Docker Container Docker Container Docker Container

              CPU Memory